In [6]:
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision.transforms import Compose
from torch.utils.data import random_split
from Utilities.dataset import EEGDataset, Compose
from Utilities.preprocessing import EEGDataProcessor

In [7]:
pp = EEGDataProcessor()

pp.DOWNSAMPLED_FREQ = 512

transforms = Compose([
    pp.correct_offset,
    pp.filter,
    pp.downsample,
    pp.normalize,
])

dataset = EEGDataset("./dataset/kub", transforms)

train_set, test_set, validation_set = random_split(dataset, [0.7, 0.2, 0.1])

train_loader = DataLoader(train_set, batch_size=32, num_workers=6, shuffle=True)
test_loader = DataLoader(test_set, batch_size=32, num_workers=6)
val_loader = DataLoader(validation_set, batch_size=32, num_workers=6)

In [8]:
len(dataset)

900

In [9]:
import torchmetrics
from torch.optim import Adam
from torch.nn.functional import binary_cross_entropy, one_hot
import lightning.pytorch as pl

class SignalCNN(pl.LightningModule):
    channels = 16
    
    def __init__(self):
        super(SignalCNN, self).__init__()
        
        self.features = nn.Sequential(
            nn.Conv1d(
                in_channels = self.channels, 
                out_channels = 32, 
                kernel_size = 11,
            ),
            nn.ReLU(),
            nn.AvgPool1d(2),
            nn.BatchNorm1d(32),
            nn.Dropout(0.3),
            
            nn.Conv1d(
                in_channels = 32, 
                out_channels = 64, 
                kernel_size = 5,
            ),
            nn.ReLU(),
            nn.AvgPool1d(3),
            nn.BatchNorm1d(64),
            nn.Dropout(0.1),
            
            nn.Conv1d(
                in_channels = 64, 
                out_channels = 128, 
                kernel_size = 3,
            ),
            nn.ReLU(),
            nn.AvgPool1d(3),
            nn.BatchNorm1d(128),
            nn.Dropout(0.1),
            
            nn.Conv1d(
                in_channels = 128, 
                out_channels = 128, 
                kernel_size = 1,
            ),
            nn.ReLU(),
            nn.AvgPool1d(5),
            nn.BatchNorm1d(128),
        )
        
        self.classifier = nn.Sequential(
            nn.Flatten(),
            
            nn.Linear(2816, 512),
            nn.BatchNorm1d(512),
            
            nn.Linear(512, 128),
            nn.BatchNorm1d(128),
            
            nn.Linear(128, 3),
            nn.Softmax(),
        )
        
        self.accuracy = torchmetrics.Accuracy(task="multiclass", num_classes=3)

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x
    
    def configure_optimizers(self):
        optimizer = Adam(self.parameters(), lr=1e-3)
        return optimizer
    
    def training_step(self, batch, batch_idx):
        data, label = batch
        
        # get predictions
        output = self(data)
        
        # convert for loss calculation
        label = one_hot(label,num_classes=3).to(torch.float32)
    
        # calculate loss
        loss = binary_cross_entropy(output, label)
        self.log("training_loss", loss, on_step=True, on_epoch=True, prog_bar=False, logger=True)
                
        # calculate accuracy
        self.accuracy(output.argmax(dim=-1), label.argmax(dim=-1))
        self.log('train_acc', self.accuracy, on_step=True, on_epoch=True, prog_bar=False, logger=True)
        
        return loss
    
    def test_step(self, batch, batch_idx):
        data, label = batch
        
        # get predictions
        output = self(data)
        
        # convert for loss calculation
        label = one_hot(label,num_classes=3).to(torch.float32)
    
        # calculate loss
        loss = binary_cross_entropy(output, label)
        self.log("test_loss", loss, on_step=True, on_epoch=True, prog_bar=False, logger=True)
                
        # calculate accuracy
        self.accuracy(output.argmax(dim=-1), label.argmax(dim=-1))
        self.log('test_acc', self.accuracy, on_step=True, on_epoch=True, prog_bar=False, logger=True)
    
    def validation_step(self, batch, batch_idx):
        data, label = batch
        
        # get predictions
        output = self(data)
        
        # convert for loss calculation
        label = one_hot(label,num_classes=3).to(torch.float32)
    
        # calculate loss
        loss = binary_cross_entropy(output, label)
        self.log("validation_loss", loss, on_step=True, on_epoch=True, prog_bar=False, logger=True)
                
        # calculate accuracy
        self.accuracy(output.argmax(dim=-1), label.argmax(dim=-1))
        self.log('validation_acc', self.accuracy, on_step=True, on_epoch=True, prog_bar=False, logger=True)

In [10]:
model = SignalCNN()

In [11]:
# from torchsummary import summary
# model.to(torch.device("cuda"))
# summary(model, (16, 2048))

# Train

In [12]:
from lightning.pytorch import Trainer
from lightning.pytorch.loggers import TensorBoardLogger

logger = TensorBoardLogger("tb_logs", name="my_eeg_cnn_model")

trainer = pl.Trainer(max_epochs=500, logger=logger)
trainer.fit(model, train_loader, val_loader)
trainer.test(model, test_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type               | Params
--------------------------------------------------
0 | features   | Sequential         | 57.9 K
1 | classifier | Sequential         | 1.5 M 
2 | accuracy   | MulticlassAccuracy | 0     
--------------------------------------------------
1.6 M     Trainable params
0         Non-trainable params
1.6 M     Total params
6.270     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/home/kacper/Documents/RoboticArmMotorImaginary/venv/lib/python3.10/site-packages/torch/nn/modules/container.py:217: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
/home/kacper/Documents/RoboticArmMotorImaginary/venv/lib/python3.10/site-packages/lightning/pytorch/loops/fit_loop.py:280: PossibleUserWarning: The number of training batches (20) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=500` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│      test_acc_epoch       │    0.8611111044883728     │
│      test_loss_epoch      │    1.0593665838241577     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss_epoch': 1.0593665838241577, 'test_acc_epoch': 0.8611111044883728}]